This script gets regulon memberships based on Dorothea

In [1]:
import os

In [2]:
import scipy, scipy.stats
import statsmodels, statsmodels.stats, statsmodels.stats.multitest

In [3]:
import pyensembl
annotation = pyensembl.EnsemblRelease(86) # better matching than version 100
annotation

EnsemblRelease(release=86, species='homo_sapiens')

# 0. user defined variables

In [4]:
DETs_input_file = '/home/adrian/projects/osteo/results/DETs_filtered/entire_DET_set.txt'
dorothea_regulons_dir = '/home/adrian/projects/osteo/data/dorothea/regulons/'
regulons_output_file = '/home/adrian/projects/osteo/results/grn/regulons.txt'

# 1. read data

## 1.1. read DETs

In [5]:
DETs = []
with open(DETs_input_file, 'r') as f:
    for line in f:
        element = line.replace('\n', '')
        DETs.append(element)
print('DETs found: {}'.format(len(DETs)))
print(len(list(set(DETs))))

DETs found: 1106
1106


In [6]:
manual_annotation = {
    'ENSG00000274619':'CFD',
    'ENSG00000278843':'MMP28',
    'ENSG00000227746':'C4A',
    'ENSG00000282854':'ASAP3',
    'ENSG00000283009':'ZNF436',
    'ENSG00000283106':'ZNF436-AS1',
    'ENSG00000263238':'CTSO',
    'ENSG00000275482':'EPHB6',
    'ENSG00000262862':'KRTAP2-3',
    'ENSG00000277909':'SPC25',
    'ENSG00000281166':'NLRP10',
    'ENSG00000280610':'KIF15',
    'ENSG00000280682':'HYOU1',
    'ENSG00000280641':'CDH4',
    'ENSG00000273707':'CDKN1C',
    'ENSG00000233450':'KIFC1',
    'ENSG00000262634':'SKA1',
    'ENSG00000275125':'PRXL2B',
    'ENSG00000281165':'SLC2A6',
    'ENSG00000276657':'PYCR3',
    'ENSG00000282147':'FAM20C'
}

In [7]:
DET_names = []
for ensembl_id in DETs:
    try:
        name = annotation.gene_name_of_gene_id(ensembl_id)
        DET_names.append(name)
        #print('\t', ensembl_id, name)
    except:
        print('Ensembl ID not found: {}'.format(ensembl_id))
        if ensembl_id in manual_annotation:
            print('\t found in manual annotation')
            name = manual_annotation[ensembl_id]
            if name in DET_names:
                print('\t already present, bypassing...')
            else:
                DET_names.append(name)
                print('\t {} appended to list of DET_names'.format(name))
        print()    
DET_names = list(set(DET_names))
print('DETs names: {}'.format(len(DET_names)))

Ensembl ID not found: ENSG00000274619
	 found in manual annotation
	 already present, bypassing...

Ensembl ID not found: ENSG00000278843
	 found in manual annotation
	 already present, bypassing...

Ensembl ID not found: ENSG00000227746
	 found in manual annotation
	 C4A appended to list of DET_names

Ensembl ID not found: ENSG00000282854
	 found in manual annotation
	 already present, bypassing...

Ensembl ID not found: ENSG00000283009
	 found in manual annotation
	 already present, bypassing...

Ensembl ID not found: ENSG00000283106
	 found in manual annotation
	 already present, bypassing...

Ensembl ID not found: ENSG00000263238
	 found in manual annotation
	 CTSO appended to list of DET_names

Ensembl ID not found: ENSG00000275482
	 found in manual annotation
	 already present, bypassing...

Ensembl ID not found: ENSG00000262862
	 found in manual annotation
	 already present, bypassing...

Ensembl ID not found: ENSG00000277909
	 found in manual annotation
	 SPC25 appended to list

## 1.2. read Dorothea regulons

In [8]:
dorothea_regulons = {}
files = os.listdir(dorothea_regulons_dir)
for file in files:
    tf = file.replace('.txt', '')
    targets = []
    with open(dorothea_regulons_dir+file, 'r') as f:
        for line in f:
            target = line.replace('\n', '')
            targets.append(target)
    dorothea_regulons[tf] = targets

# 2. analysis

# 2.1. get all TFs that regulate DETs

In [9]:
putative_TFs = []
for DET_name in DET_names:
    for tf in dorothea_regulons:
        if DET_name in dorothea_regulons[tf]:
            if tf not in putative_TFs:
                putative_TFs.append(tf)
print('found {} putative regulatory TFs'.format(len(putative_TFs)))
print(putative_TFs)

found 1189 putative regulatory TFs
['KLF16', 'IRF6', 'IRX5', 'SP6', 'ZNF710', 'E2F8', 'PRDM14', 'GRHL3', 'GRHL1', 'ZBED4', 'ZNF750', 'BNC1', 'MYCL', 'HES2', 'ZBTB7B', 'ZBTB42', 'ZBTB7C', 'SOX15', 'PLAGL2', 'HIVEP3', 'NHLH2', 'WIZ', 'MYBL1', 'CREB3L4', 'HSF4', 'MZF1', 'GATA5', 'ZNF789', 'HOXB3', 'FOXG1', 'ZNF575', 'ZSCAN30', 'FOXS1', 'NFATC3', 'NR5A2', 'ZNF785', 'ZNF76', 'IRF5', 'TBX6', 'ZGLP1', 'ZNF775', 'TBX2', 'DEAF1', 'ZFHX2', 'ZBTB48', 'ZNF518A', 'ETV2', 'ZNF182', 'ZFP82', 'MXD4', 'ZIC3', 'ZNF18', 'SP140L', 'GLI4', 'ZNF814', 'EGR4', 'SALL4', 'TRERF1', 'PKNOX2', 'TSHZ2', 'ZFPM2', 'CRX', 'HOXA5', 'MEOX2', 'RXRG', 'ISL2', 'ZNF536', 'SP1', 'ZNF711', 'SIX1', 'ZNF324', 'ZBED2', 'TSHZ3', 'JUN', 'MECOM', 'FOXC2', 'LMX1B', 'DNMT1', 'CSRNP3', 'TCF21', 'PRDM8', 'NPAS3', 'FOXA3', 'HMX3', 'RREB1', 'ZNF320', 'LHX9', 'VEZF1', 'ZNF502', 'MSX1', 'SHOX2', 'ISX', 'PRRX2', 'TCF7L1', 'VENTX', 'GFI1', 'NME2', 'MYRF', 'GLIS1', 'NR2E3', 'RUNX2', 'NKX2-3', 'NR4A2', 'SALL1', 'SHOX', 'RAX2', 'DMRT3', 'PCGF2'

# 2.2. find enriched regulons for each TF, do we have a hypergeometric enrichment?

For each TF, do we have a hypergeometric enrichment? In other words, find a disproportionate number of DETs among the TF targets.

In [10]:
# M is the population size
# n is the number of successes in the population 
# N is the sample size (previously n)
# x is still the number of drawn “successes”

# M is the total number of targets in Dorothea
# n is the number of targets of a particular TF in Dorothea
# N is the size of DETs
# x is the number of DETs that are targets of a particular TF

M = 20000
n = 200

N = 1000
x = 20


print('p-value <= ' + str(x) + ': ' + str(scipy.stats.hypergeom.cdf(x, M, n, N)))
print('p-value >= ' + str(x)  + ': ' + str(scipy.stats.hypergeom.sf(x-1, M, n, N)))

p-value <= 20: 0.9989104333089479
p-value >= 20: 0.0025301883214874205


In [11]:
# define M, the number of targets in Dorothea
all_targets = []
for regulon in dorothea_regulons:
    for target in dorothea_regulons[regulon]:
        all_targets.append(target)
M = len(list(set(all_targets)))
print('found {} targets'.format(M))

# define N, the size of DETs
N = len(DET_names)
print('found {} DETs'.format(N))

found 20300 targets
found 1089 DETs


In [12]:
# hypergeometric test
p_values_uncorrected = []
hypergeom_inputs = []

for putative_TF in putative_TFs:
    
    # define n, the number of targets of that particular TF in Dorothea
    n = len(dorothea_regulons[putative_TF])
    
    # define x, the number of DETs that are targets of that particular TF
    intersect = list(set(DET_names) & set(dorothea_regulons[putative_TF]))
    x = len(intersect)
    
    
    # test
    hypergeom_inputs.append((putative_TF, x, M, n, N, intersect))
    pvalue = scipy.stats.hypergeom.sf(x-1, M, n, N)
    p_values_uncorrected.append(pvalue)

In [13]:
# Benjamini–Hochberg correction
alternative, corrected_p_values = statsmodels.stats.multitest.fdrcorrection(p_values_uncorrected, alpha=0.1)

In [17]:
# store results
f = open(regulons_output_file, 'w')
f.write('TF\tTF targets DEGs / all TF targets\tbackground DEG rank / background TF targets rank\tTargets fold-enrichment\tUncorrected hypergeometric P value\tAdjusted P value (BH; alpha=0.1)\tObserved TF target DEGs\n')

hits = 0
for i in range(len(alternative)):
    if alternative[i] == True:
        hits = hits + 1
        
        putative_TF = hypergeom_inputs[i][0]
        n = hypergeom_inputs[i][3]
        M = hypergeom_inputs[i][2]
        x = hypergeom_inputs[i][1]
        N = hypergeom_inputs[i][4]
        intersection = hypergeom_inputs[i][5]
        
        a=x/n; b=N/M
        enrichment = a/b
        
        print('Hit {}'.format(hits))
        print('{} | comparing proportion {}/{} over {}/{}'.format(putative_TF, n, M, x, N))
        print('P value: {}'.format(p_values_uncorrected[i]))
        print('Adjusted P value {}'.format(corrected_p_values[i]))
        for element in intersection:
            print(element)
        print()
        
        # storing in a file
        intersection_string = ', '.join(intersection)
        f.write('{}\t{}/{}\t{}/{}\t{}\t{}\t{}\t{}\n'.format(putative_TF, x, n, N, M, enrichment, p_values_uncorrected[i], corrected_p_values[i], intersection_string))
        
f.close()

Hit 1
KLF16 | comparing proportion 799/20300 over 59/1089
P value: 0.00806862274074536
Adjusted P value 0.07106364769441655
NECTIN1
DDIT4
PAFAH1B3
NOP2
PRKAR1B
CILP2
PPRC1
C21orf58
FOXC2
CD3EAP
RANGAP1
UBE2S
FAM167A
RASSF7
SHB
ADAMTS10
CDKN1A
TUBB2B
SH3BP1
LMNB1
PNP
FOSL1
FJX1
C11orf91
DEPTOR
PIK3R1
SESN1
PRXL2B
GCH1
TNFRSF10D
NOP56
KIF15
SDF2L1
ANTXR2
LMCD1
TUBB3
PNRC1
WASL
S100A16
CTXN1
CRISPLD2
CDCA3
EEPD1
LRRC32
TUBB4B
CCDC86
VGLL4
DCBLD2
ABHD4
FBXO32
NCLN
SLC25A10
FAM89A
RHBDF2
ASAP3
HMGA1
LIFR
SEMA7A
PSD4

Hit 2
E2F8 | comparing proportion 740/20300 over 186/1089
P value: 2.2794327652720197e-75
Adjusted P value 2.7102455579084316e-72
NECTIN1
PDE1A
UHRF1
EXO1
SPC25
RRM2
BLM
PASK
WDR62
CDKN1A
CDC20
CCNB2
SHCBP1
BUB1B
ABCA13
IQGAP3
TRIP13
RACGAP1
KIF2C
ARHGAP11B
VRK1
KIF15
ORC6
TTK
DBF4
CHAF1A
AURKB
SKA3
SAPCD2
DTL
PBK
FAM83D
KNL1
CCNB1
CDKN3
MFGE8
TCF19
KIF20A
KIF18B
TROAP
GINS1
KIFC1
AURKA
MCM2
BUB1
NOG
CCNF
CHAF1B
E2F1
MCM10
ARHGAP11A
LMNB1
FERMT1
ESPL1
HMMR
CSTA
RRM1
PLAC8
PLK4


NFKBIZ
SERPINA3
RANGAP1
IL1R1
CD55
IL6
INHBB
CDKN1A
TFPI
PNP
EFNA1
FOSL1
CXCL2
FJX1
PELI1
RHOQ
BTG1
C1QTNF1
SOCS3
STOM
GJA1
TNFRSF10D
MT2A
CCR7
GYPC
SAT1
ANGPTL4
CDC7
CEBPB
CNTNAP1
BCL6
PFKP
ADM
PNRC1
GBE1
IL1R2
CRISPLD2
IL1RL1
IFITM1
MT1X
CCDC86
ATP6V0E2
TEAD4
GBP2
TUBB6
IFRD2
MEDAG
PIM1
FAM20C
IFITM3
SPRY1
MAP4K2
PLSCR1
SOD2
LATS2
LDLRAD3

Hit 73
AR | comparing proportion 138/20300 over 17/1089
P value: 0.0011653160542906618
Adjusted P value 0.013374398338289609
PNRC1
C1orf21
PTPRM
CCNG2
CYCS
NDRG1
EYA2
KIAA1217
IL1R1
AKR1C3
MT2A
MMP13
LIFR
CDKN1A
CDH2
ZBTB16
CDC6

Hit 74
ZNF469 | comparing proportion 506/20300 over 54/1089
P value: 1.1699693306827496e-06
Adjusted P value 2.529260971239617e-05
UHRF1
GLI2
GABARAPL1
C1R
CILP2
UCHL1
TUBA1B
SHANK1
PTGER2
SLC35E4
RAC2
SH3BP1
FJX1
SHCBP1
LOXL2
ANGPTL2
PAQR4
TUBA1C
TUBA1A
SPON1
PTGIS
FNDC1
TUBB3
TGFBI
TNFRSF1B
THBS2
SAMSN1
TNC
SCG2
GTSE1
CTXN1
FNDC10
HTRA1
MELTF
GLIPR1
EPHB2
CD14
DNMT1
SLC2A6
PBK
TUBB6
PLTP
IL11
KIF26B
SELPLG
FAM20C
ROR2
RA